# Torhout-Werchter

This notebook will guide you through your first scrape for the project. It will download all artists that were on [(Torhout or) Werchter](https://www.rockwerchter.be/en/history).

In [9]:
# ! pip install requests
# ! pip install BeautifulSoup4
# ! pip install selenium

In [10]:
import requests
from bs4 import BeautifulSoup
import re
import json

Try first without using selenium. On the URL given above you'll see a lot of festival posters (one for every year). Download all the URL's they link to.

![](files/2024-08-30-09-41-04.png)

You might think that you can generate these easily, and up to a point you are right. But back in the nineties "Werchter" was "Torhout Werchter", the same festival at two locations. So the URL's for before 1998 are different.

In [11]:
BASE_URL = "https://www.rockwerchter.be/en/history"


This actually works. This could turn out to be as easy as finding an amp for your air guitar!

Store all URL's in a list.

(And I know you like to use short variable names, we all do, but we'll be using this list for a lot of code cells to come so make sure it's long enough to actually mean something.)

In [ ]:
def get_year_urls(base_url):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the annual links
    links = soup.find_all('a', href=re.compile(r'/en/history/rock-werchter-\d{4}'))
    year_urls = [f"https://www.rockwerchter.be{link['href']}" for link in links if 'href' in link.attrs]
    return year_urls

year_urls = get_year_urls(BASE_URL)
print(year_urls)  # Confirm that URLs are being fetched correctly


['https://www.rockwerchter.be/en/history/rock-werchter-2024', 'https://www.rockwerchter.be/en/history/rock-werchter-2023', 'https://www.rockwerchter.be/en/history/rock-werchter-2022', 'https://www.rockwerchter.be/en/history/rock-werchter-2019', 'https://www.rockwerchter.be/en/history/rock-werchter-2018', 'https://www.rockwerchter.be/en/history/rock-werchter-2017', 'https://www.rockwerchter.be/en/history/rock-werchter-2016', 'https://www.rockwerchter.be/en/history/rock-werchter-2015', 'https://www.rockwerchter.be/en/history/rock-werchter-2014', 'https://www.rockwerchter.be/en/history/rock-werchter-2013', 'https://www.rockwerchter.be/en/history/rock-werchter-2012', 'https://www.rockwerchter.be/en/history/rock-werchter-2011', 'https://www.rockwerchter.be/en/history/rock-werchter-2010', 'https://www.rockwerchter.be/en/history/rock-werchter-2009', 'https://www.rockwerchter.be/en/history/rock-werchter-2008', 'https://www.rockwerchter.be/en/history/rock-werchter-2007', 'https://www.rockwercht

With this list we can visit each and every one of these pages to download the list of bands that played there. Best way to do this is using a function because you'll need to run it once for every item in the list.

Write you function that has the url as parameter and returns a list of all the bands that played there. This means not storing any of the information for which day they played on, but ...

![](files/2024-08-30-10-02-55.png)

That information is not always available. Which basically means that if a ":" is present in the text you need to ignore the text before it and split the text behind it based in ",". That would be a nice question for first year students with a little if-statement and a split. But with regex we can do this in a single line.

Your regex should also fix:
- The space before the bandnames. The band playing in 2023 is called "Stormzy", not " Stormzy".
- The non-breaking spaces you'll find in the 2018 '\xa0Angèle' and '\xa0Angus & Julia Stone'

Test using the URL for 2019 (includes dates), 2018 (no dates) and 2017 (no bands).


In [ ]:
def scrape_year_lineup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the div that contains the lineup information
    lineup_div = soup.find('div', {'data-component': 'oembed/oembed'})
    bands = []

    if lineup_div:
        # Extract text and clean data
        paragraphs = lineup_div.find_all('p')
        for paragraph in paragraphs:
            text = paragraph.get_text()
            # Ignore the date (before ":") and split by commas
            if ':' in text:
                bands_text = text.split(':', 1)[1]  # Get the part after ":"
                bands.extend([band.strip() for band in bands_text.split(',')])
    return bands




When done solving the basic problems, look at the line-up for [TW1987](https://www.rockwerchter.be/en/history/rock-torhout-werchter-1987). It contains a fun fact which although being very interesting, messes up our scraping...

![](files/2024-08-30-10-59-25.png)

Alter the function to get rid of the fun facts.

If you're happy with the function, call it for every URL you have. Save them in a dictionary (the key being the year). If you don't have any bands for that year, the key shouldn't be created.

In [ ]:
all_lineups = {}

for url in year_urls:
    year_match = re.search(r'rock-werchter-(\d{4})', url)
    if year_match:
        year = year_match.group(1)
        print(f"Scraping lineup for {year}...")
        bands = scrape_year_lineup(url)
        if bands:
            all_lineups[year] = bands

# Save the results to a JSON file
with open('./data/werchter_data.json', 'w') as f:
    json.dump(all_lineups, f, indent=4)

print("Data saved in 'data/werchter_data.json'")



Scraping lineup for 2024...
Scraping lineup for 2023...
Scraping lineup for 2022...
Scraping lineup for 2019...
Scraping lineup for 2018...
Scraping lineup for 2017...
Scraping lineup for 2016...
Scraping lineup for 2015...
Scraping lineup for 2014...
Scraping lineup for 2013...
Scraping lineup for 2012...
Scraping lineup for 2011...
Scraping lineup for 2010...
Scraping lineup for 2009...
Scraping lineup for 2008...
Scraping lineup for 2007...
Scraping lineup for 2006...
Scraping lineup for 2005...
Scraping lineup for 2004...
Scraping lineup for 2003...
Scraping lineup for 2002...
Scraping lineup for 2001...
Scraping lineup for 2000...
Scraping lineup for 1999...
Datos guardados en 'werchter_data.json'


To finish up, print all of them!

This should yield something like:

![](files/2024-09-27-10-28-02.png)

In [ ]:
for year, bands in all_lineups.items():
    print(f"Year: {year}")
    print(f"Bands: {bands}")
    print("")


Año: 2024
Bandas: ['Lenny Kravitz', 'Greta Van Fleet', 'Dropkick Murphys', 'Parkway Drive', 'The Hives', 'The Gaslight Anthem', 'STONE', "Jane's Addiction", 'PJ Harvey', 'Black Pumas', 'MEUTE', 'Eefje de Visser', 'Jalen Ngonda', 'The Streets', 'Nathaniel Rateliff & The Night Sweats', 'Slowdive', 'Johnny Marr', 'Bombay Bicycle Club', 'The Cat Empire', 'The Clockworks', 'Skindred', 'DEHD', 'Alice Merton', 'Kingfishr', 'PEUK', 'Måneskin', 'Sum 4', 'Yungblud', 'Tom Odell', 'Simple Plan', 'The Beaches', 'Frank Carter & The Rattlesnakes', 'Snow Patrol', 'dEUS', 'Tom Morello', 'Gary Clark Jr.', 'Sleaford Mods', 'Loverman', 'James Arthur', 'Archive', 'Declan McKenna', 'Glints', 'Kneecap', 'Yard Act', 'Against The Current', 'Neck Deep', 'The Armed', 'Hot Muligan', 'The Rumjacks', 'Sprints', 'Dua Lipa', 'Khruangbin', 'Avril Lavigne', 'Nothing But Thieves', 'The Kooks', 'Brihang', 'Equal Idiots', 'Róisín Murphy', 'The Blaze', 'Janelle Monáe', 'Jessie Ware', 'The Last Dinner Party', 'No Guidnce', 

Finally, store the dictionary you made in a JSON or CSV-file.

In [ ]:
with open('./data/werchter_data.json', 'w') as f:
    json.dump(all_lineups, f, indent=4)

print("Data saved in 'data/werchter_data.json'")



Datos guardados en 'data/werchter_data.json'


# Pukkelpop

Only Werchter is not enough data. Let's do [Pukkelpop](https://www.pukkelpop.be/nl/geschiedenis) as well!

This is up to you! Don't hesitate to scrape other festival's webpages as well. Think of Pinkpop, Graspop, Lowlands, ...